# Red neuronal para predecir si una persona sufrirá de un derrame cerebral

In [4]:
#----------------------------------------------------------
# Se importan las librerias
#----------------------------------------------------------

import pandas as pd

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split


#### 1. Leer el archivo stroke.csv

In [5]:
#----------------------------------------------------------
# Url de los datos. Carga local si falla el origen web
#----------------------------------------------------------

try:
    url = "https://raw.githubusercontent.com/Yurani1143/Informe_Inteligencia_Artificial/main/stroke.csv"
except:
    url = "./stroke.csv"

#----------------------------------------------------------
# Se cargan los datos
#----------------------------------------------------------
datos = pd.read_csv(url, sep=",")
datos.columns= ["sexo","edad","hipertension","enfermedades_cardiacas","casado","tipo_trabajo","tipo_residencia",
                "nivel_glucosa_promedio","indice_masa_corporal","estado_tabaquismo","derrame_cerebral"]

In [6]:
#----------------------------------------------------------
# Información de los atributos
#----------------------------------------------------------

datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   sexo                    700 non-null    object 
 1   edad                    700 non-null    float64
 2   hipertension            700 non-null    int64  
 3   enfermedades_cardiacas  700 non-null    int64  
 4   casado                  700 non-null    object 
 5   tipo_trabajo            700 non-null    object 
 6   tipo_residencia         700 non-null    object 
 7   nivel_glucosa_promedio  700 non-null    float64
 8   indice_masa_corporal    651 non-null    float64
 9   estado_tabaquismo       700 non-null    object 
 10  derrame_cerebral        700 non-null    int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 60.3+ KB


In [7]:
#----------------------------------------------------------
# Variables categóricas (tipo object)
#----------------------------------------------------------
datos.select_dtypes(include=['object']).describe()


,sexo,casado,tipo_trabajo,tipo_residencia,estado_tabaquismo
count,700,700,700,700,700
unique,2,2,5,2,4
top,Female,Yes,Private,Urban,never smoked
freq,414,522,410,356,265


In [8]:
#----------------------------------------------------------
# Variables cuantitativas (tipo entero)
#----------------------------------------------------------

datos.select_dtypes(include=['int64']).describe()

,hipertension,enfermedades_cardiacas,derrame_cerebral
count,700.000000,700.000000,700.000000
mean,0.151429,0.095714,0.355714
std,0.358722,0.294409,0.479072
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000


In [9]:
#----------------------------------------------------------
# Variables cuantitativas (tipo real)
#----------------------------------------------------------
datos.select_dtypes(include=['float64']).describe()

,edad,nivel_glucosa_promedio,indice_masa_corporal
count,700.000000,700.000000,651.000000
mean,51.485200,116.647786,29.857143
std,22.401775,53.171734,8.008309
min,0.320000,55.250000,12.000000
25%,36.000000,78.787500,24.450000
50%,55.000000,96.925000,28.500000
75%,71.000000,137.795000,34.300000
max,82.000000,271.740000,71.900000


#### 2. Seleccionar aleatoriamente el 80% del conjunto de datos para entrenar y el 20% restante para las pruebas

In [10]:
totalDatos=len(datos) # cantidad de datos
cTrain=int(totalDatos*0.8) # 80% para entrenar y 20% para probar
cTest=totalDatos-cTrain
print(totalDatos,cTrain,cTest)
train_data,test_data= train_test_split(datos, train_size=cTrain, test_size=cTest)


700 560 140


700 - Conjunto total de datos

560 - Corresponde al 80% del conjunto de entrenamiento 

140 - Corresponde al 20 % del conjunto de prueba

In [11]:
#----------------------------------------------------------
# Dimension del conjunto de entrenamiento
#----------------------------------------------------------
train_data.shape


(560, 11)

Los datos de entrenamiento son 560 y tiene 11 atributos 

In [12]:
#----------------------------------------------------------
# Muestra 5 datos de entrenamiento
#----------------------------------------------------------

train_data.head()

,sexo,edad,hipertension,enfermedades_cardiacas,casado,tipo_trabajo,tipo_residencia,nivel_glucosa_promedio,indice_masa_corporal,estado_tabaquismo,derrame_cerebral
104,Female,57.0,0,0,Yes,Govt_job,Urban,110.52,28.5,Unknown,1
547,Female,26.0,0,0,Yes,Private,Rural,75.29,22.6,smokes,0
562,Female,1.8,0,0,No,children,Urban,58.26,16.5,Unknown,0
404,Female,61.0,0,0,Yes,Self-employed,Rural,93.48,23.7,Unknown,0
99,Female,69.0,0,0,Yes,Govt_job,Urban,82.81,28.0,never smoked,1


De los 560 datos de entrenamiento se muestra 5 datos aleatorios 

#### 3. Utilizar una estrategia para normalizar los datos y llenar los datos faltantes

### Pipeline para los atributos categóricos

In [13]:
# 5 atributos categóricos
cat_attribs = ['sexo','casado','tipo_trabajo','tipo_residencia','estado_tabaquismo']

cat_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("cat_encoder", OneHotEncoder(sparse=False))
    ])

### Pipeline para los atributos númericos

In [14]:
# 5 atributos numéricos
num_attribs = ['edad','hipertension','enfermedades_cardiacas','nivel_glucosa_promedio','indice_masa_corporal']

num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())

    ])

### Pipeline completo

In [15]:
full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", cat_pipeline, cat_attribs),
])

In [16]:
#----------------------------------------------------------
# Se transforman los datos entrenados 
# Variables independientes
#----------------------------------------------------------
X_train = full_pipeline.fit_transform(train_data)
X_train.shape

(560, 20)

Los datos de entrenamiento son 560 y 20 por el OneHotEncoding (5 de los atributos numéricos + 15 de los atributoscategóricos)

In [17]:
#---------------------------------------------------------------
# Muestra los datos de entrenamiento después de la codificación
#---------------------------------------------------------------

X_train[0,:]

array([ 0.2416507 , -0.4259514 , -0.32332103, -0.11008245, -0.18026053,
        1.        ,  0.        ,  0.        ,  1.        ,  1.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.        ,  1.        ,  0.        ,  0.        ,  0.        ])

In [18]:
#----------------------------------------------------------
# Variable dependiente -> derrame_cerebral
#----------------------------------------------------------

y_train = train_data["derrame_cerebral"]
y_train

104    1
547    0
562    0
404    0
99     1
      ..
42     1
286    0
59     1
694    0
697    0
Name: derrame_cerebral, Length: 560, dtype: int64

In [19]:
#----------------------------------------------------------
# Se transforman los datos de prueba 
# Variables independientes
#--------------------------------------------------------
X_test = full_pipeline.transform(test_data) 
X_test

array([[ 0.33111073, -0.4259514 , -0.32332103, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.31517103, -0.4259514 , -0.32332103, ...,  0.        ,
         1.        ,  0.        ],
       [-1.4133598 , -0.4259514 , -0.32332103, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-1.68173989, -0.4259514 , -0.32332103, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.13625098, -0.4259514 ,  3.09290118, ...,  0.        ,
         1.        ,  0.        ],
       [-0.07145939, -0.4259514 , -0.32332103, ...,  0.        ,
         0.        ,  1.        ]])

In [20]:
#----------------------------------------------------------
# Variable dependiente -> derrame_cerebral
#----------------------------------------------------------
y_test = test_data["derrame_cerebral"]
y_test

273    0
226    1
394    0
497    0
490    0
      ..
577    0
492    0
245    1
155    1
156    1
Name: derrame_cerebral, Length: 140, dtype: int64

#### 4. Construir 5 redes neuronales variando en la topología de la red la cantidad de capas ocultas y de neuronas por cada capa oculta. Puede también variar los hiperparámetros solver y la función de activación. En todas las pruebas debe usar un random_state=123. Incluya en el notebook una tabla a manera de resumen con el accuracy obtenido en cada caso y también las matrices de confusión

In [21]:
# Se crea una lista para almacenar los resultados
resultados = []

# Define una lista de topologías a probar
topologias = [
    (10), # una capa oculta con 20 neuronas
    (10,10), # dos capas ocultas una con 20 y otra con 10 neuronas
    (20,), # una capa oculta con 30 neuronas
    (20, 20), # dos capas ocultas con 30 y otra con 20 neuronas 
    (30,), # una capa oculta con 40 neuronas
    ]

# Define una lista de funciones de activación a probar
func_activacion = ["relu", "logistic"] 

# Define una lista de solvers a probar
solvers = ["adam", "sgd"]

# Itera para cada topología
for topologia in topologias:
    # Itera para cada función de activación
    for activacion in func_activacion:
        # Itera para cada solver
        for solver in solvers:
            # Crea el modelo de red neuronal
            model = MLPClassifier(hidden_layer_sizes=topologia, activation=activacion, solver=solver, random_state=123)
            # Entrena el modelo
            model.fit(X_train, y_train)
            # Realiza las predicciones
            y_pred = model.predict(X_test)
            # Calcula el accuracy
            accuracy = round(accuracy_score(y_test, y_pred),4) # con 4 decimales
            # Calcula la matriz de confusión
            confusion = confusion_matrix(y_test, y_pred)
            # Almacena el resultado
            resultados.append({"Topología": topologia, "Función de activación": activacion, "Solver": solver, "Accuracy": accuracy, "Matriz de confusión": confusion})

# Imprime los resultados
#print(resultados)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multi

In [22]:
# Se imprime la tabla con los resultados
result = pd.DataFrame(resultados)
display(result)

,Topología,Función de activación,Solver,Accuracy,Matriz de confusión
0,10,relu,adam,0.7786,"[[77, 13], [18, 32]]"
1,10,relu,sgd,0.7929,"[[84, 6], [23, 27]]"
2,10,logistic,adam,0.7786,"[[78, 12], [19, 31]]"
3,10,logistic,sgd,0.6429,"[[90, 0], [50, 0]]"
4,"(10, 10)",relu,adam,0.7714,"[[77, 13], [19, 31]]"
5,"(10, 10)",relu,sgd,0.7214,"[[83, 7], [32, 18]]"
6,"(10, 10)",logistic,adam,0.7786,"[[74, 16], [15, 35]]"
7,"(10, 10)",logistic,sgd,0.6429,"[[90, 0], [50, 0]]"
8,"(20,)",relu,adam,0.7714,"[[75, 15], [17, 33]]"
9,"(20,)",relu,sgd,0.7857,"[[79, 11], [19, 31]]"


Con los resultados obtenidos se construyó la tabla anterior, en la cual se tiene la topología de la red, la función de activación, el hiperparámetro solver, el accuracy y la matriz de confusión.

#### 5. Indique en el notebook usando una celda de tipo Markdown los hiperparámetros que por el momento le permiten obtener la red con mayor accuracy

In [23]:
# Encontrar la red neuronal con el mayor valor de Accuracy
max_accuracy = max(resultados, key=lambda x: x["Accuracy"])

# Imprime los hiperparámetros del mejor resultado
print("-------------------------------------------")
print("Mejores hiperparámetros:")
print("-------------------------------------------")
print("Topología:", max_accuracy["Topología"])
print("Función de activación:", max_accuracy["Función de activación"])
print("Solver:", max_accuracy["Solver"])
print("Accuracy:", max_accuracy["Accuracy"])
print("Matriz de confusión:", max_accuracy["Matriz de confusión"])

-------------------------------------------
Mejores hiperparámetros:
-------------------------------------------
Topología: (20,)
Función de activación: logistic
Solver: adam
Accuracy: 0.8
Matriz de confusión: [[76 14]
 [14 36]]


Se obtiene la red neuronal con el valor más alto en accuracy y se imprime los hiperparámetros.

#### 6. Seleccione uno de los hiperparámetros disponibles en la documentación (https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) que sea diferente al solver, a la función de activación, y al random_state. Realice dos variaciones en el hiperparámetro seleccionado manteniendo los otros hiperparámetros del punto anterior. Indique el accuracy obtenido al modificar el hiperparámetro seleccionado y analice si la red mejora, empeora, o mantiene su exactitud.Incluya en el notebook dicho análisis

In [24]:
# Se crea una lista para almacenar los resultados
resultados_p7 = []

# Define una lista de tasas de aprendizaje inicial para la prueba
learning_rate_inits = [0.01, 0.001]
activation = "identity" # función de activación ReLU
solver = "lbfgs" # solver Adam
# Itera para cada tasa de aprendizaje inicial
for learning_rate_init in learning_rate_inits:

    # Define la topología de la red
    hidden_layer_sizes = (10, 10) # dos capas ocultas con 10 neuronas cada una
    # Crea el modelo de red neuronal
    model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, activation=activation, solver=solver, random_state=123, learning_rate_init=learning_rate_init)
    # Entrena el modelo
    model.fit(X_train, y_train) 
    # Realiza las predicciones
    y_pred = model.predict(X_test)
    # Calcula el accuracy
    accuracy = accuracy_score(y_test, y_pred)
    # Almacena el resultado
    resultados_p7.append({"learning_rate_init": learning_rate_init, "accuracy": accuracy})

# Imprime los resultados
print("---------------------------------")
print(resultados_p7)
print("---------------------------------")

---------------------------------
[{'learning_rate_init': 0.01, 'accuracy': 0.8142857142857143}, {'learning_rate_init': 0.001, 'accuracy': 0.8142857142857143}]
---------------------------------


In [25]:
# Se imprime la tabla con los resultados
resultado_punto7 = pd.DataFrame(resultados_p7)
display(resultado_punto7)

,learning_rate_init,accuracy
0,0.010,0.814286
1,0.001,0.814286


In [26]:
#----------------------------------------------------------
# Comparación de los accuracy del punto 5 y punto 6
#----------------------------------------------------------

# Se imprime la red neuronal con el mayor valor de Accuracy del punto 5
print("Accuracy de la red neuronal del punto 5:", max_accuracy["Accuracy"])

# Encontrar la red neuronal con el mayor valor de Accuracy
max_accuracy_p7 = max(resultados_p7, key=lambda x: x["accuracy"])

# Se imprime la red neuronal con el mayor valor de Accuracy del punto 6
print("Accuracy de la red neuronal del punto 6:", max_accuracy_p7["accuracy"])
print("    ")

if max_accuracy["Accuracy"] > max_accuracy_p7["accuracy"]:
    print("La red neuronal empeoró la exactitud, por lo que la mejor red tiene un accuracy de", round(max_accuracy["Accuracy"],4),
          "que corresponde al punto 5.")
    print("-------------------------------------------")
    print("Mejores hiperparámetros:")
    print("-------------------------------------------")
    print("Topología:", max_accuracy["Topología"])
    print("Función de activación:", max_accuracy["Función de activación"])
    print("Solver:", max_accuracy["Solver"])
    print("Accuracy:", max_accuracy["Accuracy"])
    print("Matriz de confusión:", max_accuracy["Matriz de confusión"])
    
elif max_accuracy["Accuracy"] < max_accuracy_p7["accuracy"]:
    print("La red neuronal mejoró la exactitud, por lo que la mejor red tiene un accuracy de", round(max_accuracy_p7["accuracy"],4), 
          "que corresponde al punto 7.")
    print("-------------------------------------------")
    print("Mejores hiperparámetros:")
    print("-------------------------------------------")
    print("learning_rate_init:", max_accuracy_p7["learning_rate_init"])
    print("Función de activación:", activation)
    print("Solver:", solver)
else:
    print("La red neuronal mantuvo la exactitud, ya que los valores de accuracy dieron igual aunque se haya realizado variaciones en los hiperparámetros.")



Accuracy de la red neuronal del punto 5: 0.8
Accuracy de la red neuronal del punto 6: 0.8142857142857143
    
La red neuronal mejoró la exactitud, por lo que la mejor red tiene un accuracy de 0.8143 que corresponde al punto 7.
-------------------------------------------
Mejores hiperparámetros:
-------------------------------------------
learning_rate_init: 0.01
Función de activación: identity
Solver: lbfgs
